In [ ]:
import tensorflow
import keras
from keras.applications import ResNet50
from keras.layers import GlobalAveragePooling2D, Dense
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras import Model, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

import numpy as np

In [ ]:
train_path = # train dir
valid_path = # val dir
test_path = # test dir
filepath = # for output model

In [ ]:
# A few things to keep in mind are train, validation, and test data preprocessing and some parameters.


num_train_samples = 656  # DDI

num_val_samples = 219  # my ISIC validation subset

num_test_samples = 607  # my Dermatlas test set

train_batch_size = 10

val_batch_size = 10

test_batch_size = 10

image_size = 224  # this is needed for ResNet


# Declare how many steps are needed in an iteration

train_steps = np.ceil(num_train_samples / train_batch_size)

val_steps = np.ceil(num_val_samples / val_batch_size)

test_steps = np.ceil(num_test_samples / test_batch_size)


train_batches = ImageDataGenerator(
    preprocessing_function=tensorflow.keras.applications.resnet50.preprocess_input
).flow_from_directory(
    train_path,
    target_size=(image_size, image_size),
    batch_size=train_batch_size,
    class_mode="categorical",
)


valid_batches = ImageDataGenerator(
    tensorflow.keras.applications.resnet50.preprocess_input
).flow_from_directory(
    valid_path,
    target_size=(image_size, image_size),
    batch_size=val_batch_size,
    class_mode="categorical",
)


test_batches = ImageDataGenerator(
    tensorflow.keras.applications.resnet50.preprocess_input
).flow_from_directory(
    test_path,
    target_size=(image_size, image_size),
    batch_size=test_batch_size,
    class_mode="categorical",
    shuffle=False,
)

In [ ]:
base_model = ResNet50(weights="imagenet", include_top=False)

x = base_model.output

x = GlobalAveragePooling2D()(x)

predictions = Dense(2, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)


# freezing all blocks except the last

for layer in base_model.layers[0:143]:

    layer.trainable = False


for layer in base_model.layers[143:]:

    layer.trainable = True


model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

checkpoint = ModelCheckpoint(
    filepath, monitor="val_accuracy", verbose=1, save_best_only=True, mode="max"
)


# try early stopping to avoid overfitting

earlystopping = EarlyStopping(
    monitor="val_accuracy", mode="max", patience=5, restore_best_weights=True
)


reduce_lr = ReduceLROnPlateau(
    monitor="val_accuracy",
    factor=0.5,
    patience=5,
    verbose=1,
    mode="max",
    min_lr=0.00001,
)


callbacks_list = [checkpoint, reduce_lr]

history = model.fit_generator(
    train_batches,
    steps_per_epoch=train_steps,
    validation_data=valid_batches,
    validation_steps=val_steps,
    epochs=30,  # was 20 but initial 30
    verbose=1,
    callbacks=callbacks_list,
)